## Text Analysis - Topic Modelling
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [ ]:
%%capture

import bokeh.plotting
import penelope.notebook.topic_modelling as gui
from penelope.utility import get_logger, pandas_utils

import __paths__  # pylint: disable=unused-import

logger = get_logger()

bokeh.plotting.output_notebook()
pandas_utils.set_default_options()
current_state: gui.TopicModelContainer = gui.TopicModelContainer.singleton
corpus_folder: str = "/data/westac/sou_kb_labb"

### <span style='color: green'>PREPARE</span> Load Topic Model <span style='float: right; color: red'>MANDATORY</span>

In [ ]:
gui.display_load_topic_model_gui(corpus_folder, current_state())

# FIXME: Add to load display:
# import pandas as pd
# from penelope.notebook.ipyaggrid_utility import display_grid

# def compute_topic_proportions(document_topic_weights: pd.DataFrame):
#     doc_topic_dists = document_topic_weights[['document_id', 'topic_id', 'weight', 'n_raw_tokens']]
#     # compute sum of (topic weight x document lengths)
#     topic_freqs = doc_topic_dists.assign(t_weight=lambda df: df.weight * df.n_raw_tokens).groupby('topic_id')['t_weight'].sum()
#     # normalize on total sum
#     topic_proportion = (topic_freqs / topic_freqs.sum()).sort_values(ascending=False)
#     # return global topic proportion
#     topic_proportion = pd.DataFrame(data={'topic_proportion': 100.0 * topic_proportion})
#     return topic_proportion

# document_topic_weights = current_state().inferred_topics.document_topic_weights
# topic_token_overview = current_state().inferred_topics.topic_token_overview

# topic_proportions = compute_topic_proportions(document_topic_weights)
# topic_proportions = topic_proportion.merge(topic_token_overview, left_index=True, right_index=True)
# #topic_proportion = current_state().inferred_topics.topic_token_overview
# #display(display_grid(topic_proportion))
# topic_proportions.to_excel('mallet-500.xlsx')
# #display(current_state().inferred_topics.topic_token_overview)

### <span style='color: green;'>BROWSE</span> Find topics by token<span style='color: red; float: right'>TRY IT</span>

Displays topics in which given token is among toplist of dominant words.

In [ ]:
try:
    gui.find_topic_documents_gui(
        current_state().inferred_topics.document_topic_weights, current_state().inferred_topics.topic_token_overview
    )
except Exception as ex:
    logger.exception(ex)

### <span style='color: green;'>BROWSE</span> Browse Topic Documents<span style='color: red; float: right'>TRY IT</span>

Displays documents in which a topic occurs above a given threshold.

In [ ]:
try:
    gui.display_topic_documents_gui(current_state())
except Exception as ex:
    logger.exception(ex)

### <span style='color: green;'>VISUALIZE</span> Display Topic's Word Distribution as a Wordcloud<span style='color: red; float: right'> TRY IT</span>

In [ ]:
bokeh.plotting.output_notebook()
try:
    gui.display_topic_wordcloud_gui(current_state())
except Exception as ex:
    logger.exception(ex)

### <span style='color: green;'>VISUALIZE</span> Topic-Word Distribution<span style='color: red; float: right'>TRY IT</span>


In [ ]:
try:
    gui.display_topic_word_distribution_gui(current_state())
    # topic_word_distribution_gui.display_topic_tokens(current_state(), topic_id=0, n_words=100, output_format='Chart')
except Exception as ex:
    logger.exception(ex)

### <span style='color: green;'>VISUALIZE</span> Topic Trends over Time<span style='color: red; float: right'>RUN</span>

In [ ]:
try:
    gui.display_topic_trends_gui(current_state())
except Exception as ex:
    logger.exception(ex)

### <span style='color: green;'>VISUALIZE</span> Topic Trends Overview<span style='color: red; float: right'>TRY IT</span>

- The topic shares  displayed as a scattered heatmap plot using gradient color based on topic's weight in document.
- [Stanford’s Termite software](http://vis.stanford.edu/papers/termite) uses a similar visualization.

In [ ]:
bokeh.plotting.output_notebook()
try:
    gui.display_topic_trends_overview_gui(current_state())
except ValueError as ex:
    logger.exception(ex)

### <span style='color: green;'>VISUALIZE</span> Topic Topic Network<span style='color: red; float: right'>TRY IT</span>

Computes weighted graph of topics co-occurring in the same document. Topics are defined as co-occurring in a document if they both have a weight above given threshold. The edge weights are the number of co-occurrences (binary yes or no). Node size reflects topic proportions over the entire corpus computed in accordance to LDAvis topic proportions.

In [ ]:
bokeh.plotting.output_notebook()
gui.display_topic_topic_network_gui(current_state())

### <span style='color: green;'>VISUALIZE</span> Document Topic Network<span style='color: red; float: right'>TRY IT</span>


In [ ]:
bokeh.plotting.output_notebook()
gui.display_topic_document_network_gui(current_state())

### <span style='color: green;'>VISUALIZE</span> Focus-Topic Document Network<span style='color: red; float: right'>TRY IT</span>


In [ ]:
bokeh.plotting.output_notebook()
gui.display_focus_topics_network_gui(current_state())